# Задание: Обработка данных статистических сборников

### Цель
        Привести разрозненные данные статистических сборников Росстата к форме, удобной для
    анализа, в том числе для использования в аналитических и научно-исследовательских
    целях.
### Задачи
 __1.__ Используя статистические сборники Росстата1, _определить перечень показателей_ в разрезе субъектов Российской Федерации, относящийся к тематикам «здравоохранение» и «демография».
 
__2.__ Из полученного перечня _выбрать 5-7 показателей тематики «здравоохранение» и 5-7 показателей тематики «демография»_.

__3.__ В отношении выбранных показателей _сформировать датасеты в формате временных рядов_ за максимально озможные временные периоды, в том числе:
- провести предобработку данных (работа с пропусками, нормализация
данных2
);

- определить перечень потенциальных идентификаторов (внешних ключей),
позволяющих обеспечивать взаимоувязку данных и устанавливать
отношения между датасетами;
-  учесть проблемы объединения данных по показателям (разные
географические привязки, разные уровни агрегации, изменение методологии,
различные временные промежутки сбора данных / проведения
статистического наблюдения), предложить возможные решения выявленных
проблем.

__4.__ Сформировать из обработанных датасетов базу данных на любой удобной СУБД
(MySQL, PostgreSQL, SQLite).3
<a id='section_id2'></a>

### Подберем показатели для оценки субъектов Российской Федерации исходя из нормативных документов.


### Перечень показателей «демография»:
- [__общая численность__](#section_id)__  [(Демографический ежегодник России 2019)](https://rosstat.gov.ru/bgd/regl/B19_16/Main.htm)
-  [__смертность__](#section_id3)__ ([Регионы России. Социально-экономические показатели - 2019 г.](https://rosstat.gov.ru/bgd/regl/b19_14p/Main.htm))
- [__рождаемость__](#section_id4)__ ([Регионы России. Социально-экономические показатели - 2019 г.](https://rosstat.gov.ru/bgd/regl/b19_14p/Main.htm))
- [__браки__](#section_id5)__ ([Регионы России. Социально-экономические показатели - 2019 г.](https://rosstat.gov.ru/bgd/regl/b19_14p/Main.htm))
- [__разводы__](#section_id6)__ ([Регионы России. Социально-экономические показатели - 2019 г.](https://rosstat.gov.ru/bgd/regl/b19_14p/Main.htm))
- [__миграция__](#section_id7)__ ([Регионы России. Социально-экономические показатели - 2019 г.](https://rosstat.gov.ru/bgd/regl/b19_14p/Main.htm))

### Перечень показателей «здравоохранение»:
- [__расходы консолидированных бюджетов на здравоохранение__](#section_id8)  [(Здравоохранение в России - 2019 г.)](https://rosstat.gov.ru/bgd/regl/b19_34/Main.htm)
- [__среднемесячная номинальная начисленная заработная плата работников здравоохранения__](#section_id9)  [(Здравоохранение в России - 2019 г.)](https://rosstat.gov.ru/bgd/regl/b19_34/Main.htm)
- [__число больничных коек__](#section_id10)  [(Здравоохранение в России - 2019 г.)](https://rosstat.gov.ru/bgd/regl/b19_34/Main.htm)
- [__число пациентов с алкоголизмом и алкогольными психозами__](#section_id11)  [(Здравоохранение в России - 2019 г.)](https://rosstat.gov.ru/bgd/regl/b19_34/Main.htm)
- [__прерывание беременности (аборты)__](#section_id12)([Регионы России. Социально-экономические показатели - 2019 г.](https://rosstat.gov.ru/bgd/regl/b19_14p/Main.htm))


        В ходе решения задания была изучена информация и подобраны необходимые показатели на основе которых составлялись датасеты. Для решения этой задачи использовались библиотеки pandas, numpy для работы с датафреймами и обработки данных. Библиотеки wget, zipfile, lxml ддля обработки docx документов, т.к. часть информации была не в виде xls. Библиотеки pymysql, os, sqlalchemy для записи в базу данных. К сожелению в связи с нехваткой времени не успел реализовать автоматическое добавление INDEX и FOREIGN KEY. В Dbeaver все это можно сделать, но хотелось этот процесс автоматизировать. В качестве внешних ключей хотел использовать столец 'Субъект',  т.к. это уникальное значение и его удобно использовать для связи с другми таблицами.При объединении данных проблемой было, то что они были разноформатными, поэтому пришлось использовать дополнительные функции.

In [268]:
import pandas as pd
import numpy as np
import wget
import zipfile
from lxml import etree
import pymysql
from os import environ
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, DateTime

In [269]:
# фукция для оптимизация памяти

def reduce_mem_usage(df):  
    """ Выполняет итерацию по всем столбцам фрейма данных и изменяет тип данных
        чтобы уменьшить использование памяти.       
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [270]:
# чтобы код работал необходимо изменить настройки на свои
#     art   :    1     @localhost: 3306 /healthcare_demographics
# [USERNAME]:[PASSWORD]@[DB_HOST]:[PORT]/[DB_NAME]
engine = create_engine('mysql+pymysql://art:1@localhost') # connect to server
engine.execute("CREATE DATABASE IF NOT EXISTS healthcare_demographics") #create db
engine.execute("USE healthcare_demographics")
engine = create_engine('mysql+pymysql://art:1@localhost:3306/healthcare_demographics?charset=utf8', echo=True)

Получим данные сайта __rosstat.gov.ru__ и приведем к нужному виду:

<a id='section_id'></a>

## [Население субъектов Российской Федерациина 1 января (тысяч человек)](#section_id2)

In [271]:
population = pd.read_excel("https://rosstat.gov.ru/bgd/regl/B19_16/IssWWW.exe/Stg/pril1.xls")
population.columns = ['Субъект','2000','2005','2010',
                      '2011','2012','2013','2014','2015','2016','2017','2018','2019']
population = population.drop([0,1,2,3])
population.reset_index(inplace = True, drop = True)
population.info()
population['Субъект'].tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Субъект  96 non-null     object 
 1   2000     94 non-null     float64
 2   2005     94 non-null     float64
 3   2010     94 non-null     float64
 4   2011     94 non-null     float64
 5   2012     94 non-null     float64
 6   2013     94 non-null     float64
 7   2014     94 non-null     float64
 8   2015     96 non-null     float64
 9   2016     96 non-null     float64
 10  2017     96 non-null     float64
 11  2018     96 non-null     float64
 12  2019     96 non-null     float64
dtypes: float64(12), object(1)
memory usage: 9.9+ KB


['Российская Федерация        ',
 'Центральный федеральный округ',
 'Белгородская область ',
 'Брянская область ',
 'Владимирская область ',
 'Воронежская область ',
 'Ивановская область ',
 'Калужская область ',
 'Костромская область ',
 'Курская область ',
 'Липецкая область ',
 'Московская область',
 'Орловская область ',
 'Рязанская область ',
 'Смоленская область ',
 'Тамбовская область ',
 'Тверская область ',
 'Тульская область ',
 'Ярославская область ',
 'г. Москва',
 'Северо-Западный федеральный округ',
 'Республика Карелия ',
 'Республика Коми  ',
 'Архангельская область ',
 'Ненецкий автономный округ       ',
 'Архангельская область без автономного округа',
 'Вологодская область ',
 'Калининградская область',
 'Ленинградская область',
 'Мурманская область',
 'Новгородская область ',
 'Псковская область ',
 'г. Санкт-Петербург',
 'Южный федеральный округ',
 'Республика Адыгея',
 'Республика Калмыкия',
 'Республика Крым',
 'Краснодарский край',
 'Астраханская область',
 'Волг

In [272]:
population[population['2005'].isnull()]

,Субъект,2000,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
36,Республика Крым,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1896.0,1907.0,1912.0,1914.0,1912.0
41,г. Севастополь,NaN,NaN,NaN,NaN,NaN,NaN,NaN,399.0,416.0,429.0,437.0,443.0


### До 2015 года Крым не входил в состав России поэтому во всех DataFrame в соответствующих полях будем проставлять нули.

In [273]:
population.fillna(0, inplace=True)
population.iloc[36:49]

,Субъект,2000,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
36,Республика Крым,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1896.0,1907.0,1912.0,1914.0,1912.0
37,Краснодарский край,5133.0,5119.0,5214.0,5230.0,5284.0,5330.0,5404.0,5454.0,5514.0,5571.0,5603.0,5648.0
38,Астраханская область,1012.0,1007.0,1011.0,1010.0,1015.0,1014.0,1017.0,1021.0,1019.0,1019.0,1017.0,1014.0
39,Волгоградская область,2739.0,2656.0,2614.0,2607.0,2595.0,2583.0,2569.0,2557.0,2546.0,2535.0,2521.0,2508.0
40,Ростовская область,4455.0,4353.0,4285.0,4275.0,4260.0,4254.0,4246.0,4242.0,4236.0,4231.0,4221.0,4203.0
41,г. Севастополь,0.0,0.0,0.0,0.0,0.0,0.0,0.0,399.0,416.0,429.0,437.0,443.0
42,Северо-Кавказский федеральный округ,8647.0,8994.0,9353.0,9439.0,9493.0,9541.0,9590.0,9659.0,9718.0,9776.0,9823.0,9867.0
43,Республика Дагестан,2443.0,2653.0,2869.0,2914.0,2931.0,2946.0,2964.0,2990.0,3015.0,3042.0,3064.0,3086.0
44,Республика Ингушетия,340.0,434.0,412.0,415.0,430.0,442.0,453.0,464.0,473.0,481.0,488.0,498.0
45,Кабардино-Балкарская Республика,879.0,877.0,859.0,860.0,859.0,859.0,859.0,861.0,862.0,865.0,865.0,866.0


In [274]:
reduce_mem_usage(population)

# посмотрим остались ли какие либо выпадающие значения
population.describe()

Memory usage of dataframe is 0.01 MB
Memory usage after optimization is: 0.01 MB
Decreased by 19.3%


,2000,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,4638.312500,4541.562500,4511.614746,4512.760254,4519.177246,4528.687500,4538.947754,4620.468750,4629.416504,4637.895996,4640.489746,4637.604004
std,15729.838867,15410.339844,15315.770508,15320.930664,15341.768555,15374.931641,15410.500977,15672.780273,15704.108398,15732.250977,15741.898438,15732.907227
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,43.000000,44.000000,44.000000,44.000000,44.000000
25%,909.500000,868.250000,826.250000,820.250000,812.750000,807.750000,801.000000,809.750000,806.750000,806.500000,803.250000,794.750000
50%,1376.000000,1320.500000,1283.500000,1275.000000,1276.500000,1273.000000,1270.000000,1288.000000,1284.500000,1281.500000,1274.500000,1265.000000
75%,2775.500000,2743.750000,2774.000000,2767.250000,2760.000000,2754.250000,2749.000000,2760.000000,2772.000000,2786.000000,2792.000000,2793.500000
max,146890.000000,143801.000000,142833.000000,142865.000000,143056.000000,143347.000000,143667.000000,146267.000000,146545.000000,146804.000000,146880.000000,146781.000000


In [275]:
# запишем полученный датафрейм в базу данных
population.to_sql('population', engine, if_exists='replace', index=False, chunksize=1000)

2020-09-01 22:28:20,701 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2020-09-01 22:28:20,701 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:20,704 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2020-09-01 22:28:20,704 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:20,707 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2020-09-01 22:28:20,708 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:20,710 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2020-09-01 22:28:20,710 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:20,714 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2020-09-01 22:28:20,714 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:20,719 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2020-09-01 22:28:20,723 INFO sqlalchemy.engine.base.E

<a id='section_id3'></a>

  ### <center><h2>[ОБЩИЕ КОЭФФИЦИЕНТЫ СМЕРТНОСТИ](#section_id2)   </h2> </center> 
                                                                                              
### <center><h4>(число умерших на 1000 человек населения)</h4></center>


In [276]:
def read_docx(url, **kwargs):
    """Чтение таблиц как фреймов данных из документа Word
    """
    
    docx_file = wget.download(url)
    ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}
    with zipfile.ZipFile(docx_file).open('word/document.xml') as f:
        root = etree.parse(f)
    for el in root.xpath('//w:tbl', namespaces=ns):
        el.tag = 'table'
    for el in root.xpath('//w:tr', namespaces=ns):
        el.tag = 'tr'
    for el in root.xpath('//w:tc', namespaces=ns):
        el.tag = 'td'
    return pd.read_html(etree.tostring(root), **kwargs)

In [277]:
# скачаем файл и передадим его в функцию которая его обработает
mortality_url = 'https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/02-09.docx'
mortality = read_docx(mortality_url)

# из полученных файлов сделаем датафреймы и и объединим их
df_0 = pd.DataFrame(mortality[0])
df_1 = pd.DataFrame(mortality[1])
mortality = df_0.append(df_1).reset_index(drop=True)

#обозначим названия столбцов и удалим ненужные строки
mortality.columns = ['Субъект','2005','2010','2011','2012','2013','2014', '2015','2016','2017','2018',
                     'Место, занимаемое в Российской Федерации 2018']
mortality.drop([0,25,52,72,100], inplace = True)
mortality.drop('Место, занимаемое в Российской Федерации 2018', axis=1, inplace=True)
mortality.reset_index(inplace = True, drop = True)
population['Субъект'].tolist()

['Российская Федерация        ',
 'Центральный федеральный округ',
 'Белгородская область ',
 'Брянская область ',
 'Владимирская область ',
 'Воронежская область ',
 'Ивановская область ',
 'Калужская область ',
 'Костромская область ',
 'Курская область ',
 'Липецкая область ',
 'Московская область',
 'Орловская область ',
 'Рязанская область ',
 'Смоленская область ',
 'Тамбовская область ',
 'Тверская область ',
 'Тульская область ',
 'Ярославская область ',
 'г. Москва',
 'Северо-Западный федеральный округ',
 'Республика Карелия ',
 'Республика Коми  ',
 'Архангельская область ',
 'Ненецкий автономный округ       ',
 'Архангельская область без автономного округа',
 'Вологодская область ',
 'Калининградская область',
 'Ленинградская область',
 'Мурманская область',
 'Новгородская область ',
 'Псковская область ',
 'г. Санкт-Петербург',
 'Южный федеральный округ',
 'Республика Адыгея',
 'Республика Калмыкия',
 'Республика Крым',
 'Краснодарский край',
 'Астраханская область',
 'Волг

In [278]:
# проверим пропущенные значения
mortality.isna().sum() 
mortality.head(60)

,Субъект,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Российская Федерация,161.0,142.0,135.0,133.0,130.0,131.0,130.0,129.0,124.0,125.0
1,Центральный федеральный округ,171.0,152.0,139.0,139.0,136.0,136.0,135.0,135.0,129.0,129.0
2,Белгородская область,160.0,144.0,141.0,140.0,138.0,140.0,139.0,139.0,135.0,135.0
3,Брянская область,198.0,170.0,161.0,162.0,159.0,160.0,158.0,156.0,153.0,152.0
4,Владимирская область,202.0,180.0,171.0,167.0,167.0,166.0,165.0,165.0,157.0,160.0
5,Воронежская область,185.0,170.0,159.0,156.0,155.0,157.0,153.0,152.0,146.0,147.0
6,Ивановская область,220.0,184.0,169.0,169.0,163.0,164.0,160.0,161.0,159.0,161.0
7,Калужская область,191.0,165.0,154.0,158.0,152.0,153.0,150.0,150.0,148.0,149.0
8,Костромская область,213.0,177.0,166.0,161.0,161.0,158.0,160.0,156.0,149.0,149.0
9,Курская область,198.0,176.0,168.0,166.0,161.0,166.0,163.0,161.0,155.0,154.0


In [279]:
mortality[mortality['2005'].isnull()]

,Субъект,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
36,Республика Крым,NaN,NaN,NaN,NaN,NaN,153.0,153.0,152.0,144.0,141.0
41,г. Севастополь,NaN,NaN,NaN,NaN,NaN,143.0,149.0,139.0,131.0,128.0


In [280]:
mortality.fillna(0, inplace=True)
mortality.iloc[36:42]

,Субъект,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
36,Республика Крым,0.0,0.0,0.0,0.0,0.0,153.0,153.0,152.0,144.0,141.0
37,Краснодарский край,156.0,135.0,135.0,132.0,128.0,129.0,130.0,129.0,125.0,120.0
38,Астраханская область,154.0,135.0,130.0,127.0,122.0,126.0,123.0,120.0,114.0,116.0
39,Волгоградская область,157.0,147.0,138.0,136.0,135.0,137.0,138.0,137.0,131.0,133.0
40,Ростовская область,158.0,147.0,143.0,140.0,138.0,141.0,139.0,139.0,134.0,135.0
41,г. Севастополь,0.0,0.0,0.0,0.0,0.0,143.0,149.0,139.0,131.0,128.0


In [281]:
reduce_mem_usage(mortality)

# посмотрим остались ли какие либо выпадающие значения
mortality.describe()

Memory usage of dataframe is 0.01 MB
Memory usage after optimization is: 0.01 MB
Decreased by 13.8%


,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.00000,96.00000
mean,156.864578,138.906250,131.979172,130.322922,127.520836,130.708328,130.375000,128.687500,124.12500,124.62500
std,44.231121,37.061836,34.307884,34.137028,33.809986,28.236938,28.408766,28.449129,27.39122,27.59796
min,0.000000,0.000000,0.000000,0.000000,0.000000,35.000000,33.000000,33.000000,32.00000,31.00000
25%,144.500000,129.750000,123.750000,121.500000,117.750000,118.500000,119.000000,116.750000,114.00000,114.00000
50%,163.500000,144.000000,139.000000,137.000000,134.000000,134.000000,135.000000,133.000000,129.00000,129.00000
75%,181.250000,158.250000,149.000000,149.500000,146.000000,146.500000,149.000000,148.000000,141.00000,142.50000
max,246.000000,212.000000,194.000000,196.000000,186.000000,184.000000,182.000000,179.000000,174.00000,172.00000


In [282]:
# запишем полученный датафрейм в базу данных
mortality.to_sql('mortality', engine, if_exists='replace', index=False, chunksize=1000)

2020-09-01 22:28:22,997 INFO sqlalchemy.engine.base.Engine DESCRIBE `mortality`
2020-09-01 22:28:22,998 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:23,001 INFO sqlalchemy.engine.base.Engine DESCRIBE `mortality`
2020-09-01 22:28:23,002 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:23,005 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `healthcare_demographics`
2020-09-01 22:28:23,006 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:23,011 INFO sqlalchemy.engine.base.Engine SHOW CREATE TABLE `mortality`
2020-09-01 22:28:23,012 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:23,018 INFO sqlalchemy.engine.base.Engine 
DROP TABLE mortality
2020-09-01 22:28:23,019 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:23,032 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-01 22:28:23,036 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE mortality (
	`Субъект` TEXT, 
	`2005` FLOAT(23), 
	`2010` FLOAT(23), 
	`2011` FLOAT(23), 
	`2012` FLOAT(23), 
	`20

<a id='section_id4'></a>

  ### <center><h2>[ОБЩИЕ КОЭФФИЦИЕНТЫ РОЖДАЕМОСТИ](#section_id2)   </h2> </center> 
                                                                                              
### <center><h4>(число родившихся на 1000 человек населения)</h4></center>


In [283]:
# скачаем файл и передадим его в функцию которая его обработает
birthrate_url = 'https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/02-08.docx'
birthrate = read_docx(birthrate_url)

# из полученных файлов сделаем датафреймы и и объединим их
df_0 = pd.DataFrame(birthrate[0])
df_1 = pd.DataFrame(birthrate[1])
birthrate = df_0.append(df_1).reset_index(drop=True)

#обозначим названия столбцов и удалим ненужные строки
birthrate.columns = ['Субъект','2005','2010','2011','2012','2013','2014', '2015','2016','2017','2018',
                     'Место, занимаемое в Российской Федерации 2018']
birthrate.drop([0,25,52,72,100], inplace = True)
birthrate.drop('Место, занимаемое в Российской Федерации 2018', axis=1, inplace=True)
birthrate.reset_index(inplace = True, drop = True)
population['Субъект'].tolist()

['Российская Федерация        ',
 'Центральный федеральный округ',
 'Белгородская область ',
 'Брянская область ',
 'Владимирская область ',
 'Воронежская область ',
 'Ивановская область ',
 'Калужская область ',
 'Костромская область ',
 'Курская область ',
 'Липецкая область ',
 'Московская область',
 'Орловская область ',
 'Рязанская область ',
 'Смоленская область ',
 'Тамбовская область ',
 'Тверская область ',
 'Тульская область ',
 'Ярославская область ',
 'г. Москва',
 'Северо-Западный федеральный округ',
 'Республика Карелия ',
 'Республика Коми  ',
 'Архангельская область ',
 'Ненецкий автономный округ       ',
 'Архангельская область без автономного округа',
 'Вологодская область ',
 'Калининградская область',
 'Ленинградская область',
 'Мурманская область',
 'Новгородская область ',
 'Псковская область ',
 'г. Санкт-Петербург',
 'Южный федеральный округ',
 'Республика Адыгея',
 'Республика Калмыкия',
 'Республика Крым',
 'Краснодарский край',
 'Астраханская область',
 'Волг

In [284]:
# проверим пропущенные значения
birthrate.isna().sum() 

Субъект    0
2005       2
2010       2
2011       2
2012       2
2013       2
2014       0
2015       0
2016       0
2017       0
2018       0
dtype: int64

In [285]:
birthrate[birthrate['2005'].isnull()]

,Субъект,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
36,Республика Крым,NaN,NaN,NaN,NaN,NaN,129.0,126.0,120.0,109.0,106.0
41,г. Севастополь,NaN,NaN,NaN,NaN,NaN,126.0,135.0,127.0,112.0,101.0


In [286]:
birthrate.fillna(0, inplace=True)
birthrate.iloc[36:42]

,Субъект,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
36,Республика Крым,0.0,0.0,0.0,0.0,0.0,129.0,126.0,120.0,109.0,106.0
37,Краснодарский край,102.0,122.0,122.0,130.0,131.0,135.0,135.0,132.0,120.0,115.0
38,Астраханская область,121.0,142.0,142.0,151.0,148.0,149.0,145.0,140.0,121.0,116.0
39,Волгоградская область,94.0,113.0,112.0,117.0,115.0,116.0,115.0,112.0,100.0,94.0
40,Ростовская область,91.0,109.0,109.0,117.0,117.0,121.0,121.0,116.0,103.0,97.0
41,г. Севастополь,0.0,0.0,0.0,0.0,0.0,126.0,135.0,127.0,112.0,101.0


In [287]:
reduce_mem_usage(birthrate)

# посмотрим остались ли какие либо выпадающие значения
birthrate.describe()

Memory usage of dataframe is 0.01 MB
Memory usage after optimization is: 0.01 MB
Decreased by 13.8%


,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,106.302086,129.385422,129.625000,135.625000,134.187500,137.250000,135.531250,130.781250,116.895836,110.729164
std,30.147051,38.854488,39.219021,35.904919,34.577515,27.712812,24.379425,22.441586,22.658321,21.812347
min,0.000000,0.000000,0.000000,0.000000,0.000000,91.000000,90.000000,92.000000,83.000000,76.000000
25%,90.750000,109.750000,110.000000,117.000000,116.750000,119.000000,119.000000,116.750000,103.750000,97.750000
50%,104.000000,125.000000,125.500000,132.000000,131.000000,132.500000,133.000000,128.500000,112.500000,106.000000
75%,114.250000,141.000000,141.000000,148.250000,147.000000,147.000000,144.250000,139.000000,124.000000,117.000000
max,251.000000,299.000000,290.000000,267.000000,261.000000,253.000000,238.000000,232.000000,220.000000,207.000000


In [288]:
# запишем полученный датафрейм в базу данных
birthrate.to_sql('birthrate', engine, if_exists='replace', index=True, chunksize=1000)

2020-09-01 22:28:25,465 INFO sqlalchemy.engine.base.Engine DESCRIBE `birthrate`
2020-09-01 22:28:25,466 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:25,470 INFO sqlalchemy.engine.base.Engine DESCRIBE `birthrate`
2020-09-01 22:28:25,471 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:25,475 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `healthcare_demographics`
2020-09-01 22:28:25,476 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:25,480 INFO sqlalchemy.engine.base.Engine SHOW CREATE TABLE `birthrate`
2020-09-01 22:28:25,481 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:25,485 INFO sqlalchemy.engine.base.Engine 
DROP TABLE birthrate
2020-09-01 22:28:25,485 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:25,498 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-01 22:28:25,501 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE birthrate (
	`index` BIGINT, 
	`Субъект` TEXT, 
	`2005` FLOAT(23), 
	`2010` FLOAT(23), 
	`2011` FLOAT(23), 
	`2012

<a id='section_id5'></a>

  ### <center><h2>[ОБЩИЕ КОЭФФИЦИЕНТЫ БРАЧНОСТИ](#section_id2)   </h2> </center> 
                                                                                              
### <center><h4>( на 1000 человек населения)</h4></center>

In [289]:
# скачаем файл и передадим его в функцию которая его обработает

marriage_rate_url = 'https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/02-16.docx'
marriage_rate = read_docx(marriage_rate_url)

# из полученных файлов сделаем датафреймы и и объединим их
df_0 = pd.DataFrame(marriage_rate[0])
df_1 = pd.DataFrame(marriage_rate[1])
marriage_rate = df_0.append(df_1).reset_index(drop=True)

#обозначим названия столбцов и удалим ненужные строки
marriage_rate.columns = ['Субъект','2005','2010','2011','2012','2013','2014', '2015','2016','2017','2018']
marriage_rate.drop([0,25,52,72,100], inplace = True)
marriage_rate.reset_index(inplace = True, drop = True)
marriage_rate['Субъект'].tolist()

['Российская Федерация',
 'Центральный федеральный округ',
 'Белгородская область',
 'Брянская область',
 'Владимирская область',
 'Воронежская область',
 'Ивановская область',
 'Калужская область',
 'Костромская область',
 'Курская область',
 'Липецкая область',
 'Московская область',
 'Орловская область',
 'Рязанская область',
 'Смоленская область',
 'Тамбовская область',
 'Тверская область',
 'Тульская область',
 'Ярославская область',
 'г. Москва',
 'Северо-Западный федеральный округ',
 'Республика Карелия',
 'Республика Коми',
 'Архангельская область',
 'Ненецкий автономный округ',
 'Архангельская область без автономного округа',
 'Вологодская область',
 'Калининградская область',
 'Ленинградская область',
 'Мурманская область',
 'Новгородская область',
 'Псковская область',
 'г. Санкт-Петербург',
 'Южный федеральный округ',
 'Республика Адыгея',
 'Республика Калмыкия',
 'Республика Крым',
 'Краснодарский край',
 'Астраханская область',
 'Волгоградская область',
 'Ростовская облас

In [290]:
# проверим пропущенные значения
marriage_rate.isna().sum() 

Субъект    0
2005       2
2010       2
2011       2
2012       2
2013       2
2014       0
2015       0
2016       0
2017       0
2018       0
dtype: int64

In [291]:
marriage_rate[marriage_rate['2005'].isnull()]

,Субъект,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
36,Республика Крым,NaN,NaN,NaN,NaN,NaN,87.0,92.0,71.0,75.0,61.0
41,г. Севастополь,NaN,NaN,NaN,NaN,NaN,110.0,115.0,95.0,100.0,76.0


In [292]:
marriage_rate.fillna(0, inplace=True)
marriage_rate.iloc[36:42]

,Субъект,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
36,Республика Крым,0.0,0.0,0.0,0.0,0.0,87.0,92.0,71.0,75.0,61.0
37,Краснодарский край,75.0,86.0,96.0,86.0,90.0,89.0,84.0,72.0,80.0,69.0
38,Астраханская область,77.0,87.0,91.0,86.0,85.0,82.0,74.0,66.0,66.0,56.0
39,Волгоградская область,70.0,77.0,83.0,79.0,77.0,75.0,70.0,60.0,65.0,52.0
40,Ростовская область,71.0,80.0,89.0,78.0,82.0,81.0,78.0,63.0,70.0,59.0
41,г. Севастополь,0.0,0.0,0.0,0.0,0.0,110.0,115.0,95.0,100.0,76.0


In [293]:
reduce_mem_usage(marriage_rate)

# посмотрим остались ли какие либо выпадающие значения
marriage_rate.describe()

Memory usage of dataframe is 0.01 MB
Memory usage after optimization is: 0.01 MB
Decreased by 13.8%


,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,73.239586,84.312500,90.385414,82.937500,82.864586,82.468750,77.520836,65.656250,70.062500,59.479168
std,13.328041,15.069182,16.161921,15.448258,15.503900,10.163871,10.350672,9.146559,10.018732,8.039351
min,0.000000,0.000000,0.000000,0.000000,0.000000,48.000000,42.000000,38.000000,41.000000,38.000000
25%,70.000000,80.000000,86.000000,78.750000,80.000000,78.000000,73.750000,60.750000,65.000000,55.000000
50%,74.000000,85.000000,93.000000,84.000000,84.000000,83.000000,78.000000,65.500000,71.000000,60.000000
75%,79.000000,90.000000,98.000000,90.000000,91.000000,88.250000,83.000000,71.250000,76.000000,64.250000
max,97.000000,113.000000,120.000000,109.000000,108.000000,111.000000,115.000000,96.000000,100.000000,82.000000


In [294]:
# запишем полученный датафрейм в базу данных
marriage_rate.to_sql('marriage_rate', engine, if_exists='replace', index=False, chunksize=1000)

2020-09-01 22:28:27,812 INFO sqlalchemy.engine.base.Engine DESCRIBE `marriage_rate`
2020-09-01 22:28:27,814 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:27,818 INFO sqlalchemy.engine.base.Engine DESCRIBE `marriage_rate`
2020-09-01 22:28:27,819 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:27,824 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `healthcare_demographics`
2020-09-01 22:28:27,827 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:27,832 INFO sqlalchemy.engine.base.Engine SHOW CREATE TABLE `marriage_rate`
2020-09-01 22:28:27,832 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:27,838 INFO sqlalchemy.engine.base.Engine 
DROP TABLE marriage_rate
2020-09-01 22:28:27,839 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:27,863 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-01 22:28:27,869 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE marriage_rate (
	`Субъект` TEXT, 
	`2005` FLOAT(23), 
	`2010` FLOAT(23), 
	`2011` FLOAT(23), 
	`20

<a id='section_id6'></a>

  ### <center><h2>[ОБЩИЕ КОЭФФИЦИЕНТЫ РАЗВОДИМОСТИ](#section_id2)   </h2> </center> 
                                                                                              
### <center><h4>( на 1000 человек населения)</h4></center>

In [295]:
# скачаем файл и передадим его в функцию которая его обработает

divorce_rate_url = 'https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/02-17.docx'
divorce_rate = read_docx(divorce_rate_url)

# из полученных файлов сделаем датафреймы и и объединим их
df_0 = pd.DataFrame(divorce_rate[0])
df_1 = pd.DataFrame(divorce_rate[1])
divorce_rate = df_0.append(df_1).reset_index(drop=True)

#обозначим названия столбцов и удалим ненужные строки
divorce_rate.columns = ['Субъект','2005','2010','2011','2012','2013','2014', '2015','2016','2017','2018']
divorce_rate.drop([0,25,52,72,100], inplace = True)
divorce_rate.reset_index(inplace = True, drop = True)
divorce_rate['Субъект'].tolist()

['Российская Федерация',
 'Центральный федеральный округ',
 'Белгородская область',
 'Брянская область',
 'Владимирская область',
 'Воронежская область',
 'Ивановская область',
 'Калужская область',
 'Костромская область',
 'Курская область',
 'Липецкая область',
 'Московская область',
 'Орловская область',
 'Рязанская область',
 'Смоленская область',
 'Тамбовская область',
 'Тверская область',
 'Тульская область',
 'Ярославская область',
 'г. Москва',
 'Северо-Западный федеральный округ',
 'Республика Карелия',
 'Республика Коми',
 'Архангельская область',
 'Ненецкий автономный округ',
 'Архангельская область без автономного округа',
 'Вологодская область',
 'Калининградская область',
 'Ленинградская область',
 'Мурманская область',
 'Новгородская область',
 'Псковская область',
 'г. Санкт-Петербург',
 'Южный федеральный округ',
 'Республика Адыгея',
 'Республика Калмыкия',
 'Республика Крым',
 'Краснодарский край',
 'Астраханская область',
 'Волгоградская область',
 'Ростовская облас

In [296]:
# проверим пропущенные значения
divorce_rate.isna().sum() 

Субъект    0
2005       2
2010       2
2011       2
2012       2
2013       2
2014       0
2015       0
2016       0
2017       0
2018       0
dtype: int64

In [297]:
divorce_rate[divorce_rate['2005'].isnull()]

,Субъект,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
36,Республика Крым,NaN,NaN,NaN,NaN,NaN,27.0,30.0,33.0,35.0,36.0
41,г. Севастополь,NaN,NaN,NaN,NaN,NaN,28.0,40.0,45.0,45.0,44.0


In [298]:
divorce_rate.fillna(0, inplace=True)
divorce_rate.iloc[36:42]

,Субъект,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
36,Республика Крым,0.0,0.0,0.0,0.0,0.0,27.0,30.0,33.0,35.0,36.0
37,Краснодарский край,43.0,45.0,47.0,46.0,50.0,50.0,46.0,46.0,45.0,38.0
38,Астраханская область,41.0,50.0,52.0,49.0,52.0,51.0,43.0,42.0,41.0,41.0
39,Волгоградская область,40.0,42.0,46.0,44.0,48.0,47.0,41.0,41.0,41.0,39.0
40,Ростовская область,42.0,47.0,48.0,39.0,42.0,48.0,43.0,43.0,43.0,41.0
41,г. Севастополь,0.0,0.0,0.0,0.0,0.0,28.0,40.0,45.0,45.0,44.0


In [299]:
reduce_mem_usage(divorce_rate)

# посмотрим остались ли какие либо выпадающие значения
divorce_rate.describe()

Memory usage of dataframe is 0.01 MB
Memory usage after optimization is: 0.01 MB
Decreased by 13.8%


,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,40.947918,44.677082,46.979168,45.343750,46.718750,48.541668,42.208332,42.010418,41.906250,40.156250
std,12.737321,12.435513,12.821016,12.354269,12.657183,11.106107,9.265575,9.069899,9.028701,8.675832
min,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,8.000000,9.000000,8.000000,5.000000
25%,37.750000,42.000000,44.000000,43.000000,43.750000,46.000000,40.000000,39.000000,40.000000,38.000000
50%,42.000000,46.000000,48.000000,47.000000,48.500000,50.000000,43.000000,43.000000,43.000000,41.000000
75%,47.000000,50.000000,52.000000,51.000000,54.000000,55.000000,46.250000,46.000000,46.250000,45.000000
max,71.000000,71.000000,71.000000,69.000000,70.000000,69.000000,62.000000,64.000000,60.000000,56.000000


In [300]:
# запишем полученный датафрейм в базу данных
divorce_rate.to_sql('divorce_rate', engine, if_exists='replace', index=False, chunksize=1000)

2020-09-01 22:28:30,182 INFO sqlalchemy.engine.base.Engine DESCRIBE `divorce_rate`
2020-09-01 22:28:30,182 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:30,187 INFO sqlalchemy.engine.base.Engine DESCRIBE `divorce_rate`
2020-09-01 22:28:30,188 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:30,192 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `healthcare_demographics`
2020-09-01 22:28:30,193 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:30,197 INFO sqlalchemy.engine.base.Engine SHOW CREATE TABLE `divorce_rate`
2020-09-01 22:28:30,197 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:30,202 INFO sqlalchemy.engine.base.Engine 
DROP TABLE divorce_rate
2020-09-01 22:28:30,203 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:30,217 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-01 22:28:30,222 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE divorce_rate (
	`Субъект` TEXT, 
	`2005` FLOAT(23), 
	`2010` FLOAT(23), 
	`2011` FLOAT(23), 
	`2012` F

<a id='section_id7'></a>

  ### <center><h2>[КОЭФФИЦИЕНТЫ МИГРАЦИОННОГО ПРИРОСТА)](#section_id2)   </h2> </center> 
                                                                                              
### <center><h4>(  на 10 000 человек населения)</h4></center>

In [301]:
# скачаем файл и передадим его в функцию которая его обработает

migration_url = 'https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/02-19.docx'
migration = read_docx(migration_url)

# из полученных файлов сделаем датафреймы и и объединим их
df_0 = pd.DataFrame(migration[0])
df_1 = pd.DataFrame(migration[1])
migration = df_0.append(df_1).reset_index(drop=True)

#обозначим названия столбцов и удалим ненужные строки
migration.columns = ['Субъект','2005','2010','2011','2012','2013','2014', '2015','2016','2017','2018']
migration.drop([0,25,52,72,100], inplace = True)
migration.reset_index(inplace = True, drop = True)
migration['Субъект'].tolist()

['Российская Федерация',
 'Центральный федеральный округ',
 'Белгородская область',
 'Брянская область',
 'Владимирская область',
 'Воронежская область',
 'Ивановская область',
 'Калужская область',
 'Костромская область',
 'Курская область',
 'Липецкая область',
 'Московская область',
 'Орловская область',
 'Рязанская область',
 'Смоленская область',
 'Тамбовская область',
 'Тверская область',
 'Тульская область',
 'Ярославская область',
 'г. Москва',
 'Северо-Западный федеральный округ',
 'Республика Карелия',
 'Республика Коми',
 'Архангельская область',
 'Ненецкий автономный округ',
 'Архангельская область без автономного округа',
 'Вологодская область',
 'Калининградская область',
 'Ленинградская область',
 'Мурманская область',
 'Новгородская область',
 'Псковская область',
 'г. Санкт-Петербург',
 'Южный федеральный округ',
 'Республика Адыгея',
 'Республика Калмыкия',
 'Республика Крым',
 'Краснодарский край',
 'Астраханская область',
 'Волгоградская область',
 'Ростовская облас

In [302]:
# проверим пропущенные значения
migration.isna().sum() 

Субъект    0
2005       2
2010       2
2011       2
2012       2
2013       2
2014       0
2015       0
2016       0
2017       0
2018       0
dtype: int64

In [303]:
migration[migration['2005'].isnull()]

,Субъект,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
36,Республика Крым,NaN,NaN,NaN,NaN,NaN,…,86.0,58.0,43.0,25.0
41,г. Севастополь,NaN,NaN,NaN,NaN,NaN,…,439.0,308.0,202.0,176.0


In [304]:
migration.fillna(0, inplace=True)
migration.iloc[36:42]

,Субъект,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
36,Республика Крым,0.0,0.0,0.0,0.0,0.0,…,86.0,58.0,43.0,25.0
37,Краснодарский край,70.0,44.0,117.0,87.0,135.0,84,105.0,101.0,63.0,85.0
38,Астраханская область,-6.0,-20.0,39.0,-36.0,2.0,24,-49.0,-17.0,-20.0,-34.0
39,Волгоградская область,3.0,8.0,-22.0,-27.0,-34.0,-25,-22.0,-18.0,-23.0,-15.0
40,Ростовская область,21.0,16.0,-1.0,9.0,-3.0,12,4.0,12.0,5.0,-6.0
41,г. Севастополь,0.0,0.0,0.0,0.0,0.0,…,439.0,308.0,202.0,176.0


In [305]:
reduce_mem_usage(migration)

# посмотрим остались ли какие либо выпадающие значения
migration.describe()

Memory usage of dataframe is 0.01 MB
Memory usage after optimization is: 0.01 MB
Decreased by -19.9%


,2005,2010,2011,2012,2013,2015,2016,2017,2018
count,96.00000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,-18.81250,-14.281250,2.072917,-6.500000,-10.458333,-4.427083,2.468750,-7.000000,-13.197917
std,86.34166,57.875504,61.720985,59.158306,61.887653,69.914360,56.048275,53.065998,57.605675
min,-499.00000,-174.000000,-125.000000,-138.000000,-150.000000,-223.000000,-103.000000,-132.000000,-187.000000
25%,-49.50000,-46.000000,-33.250000,-36.250000,-38.000000,-30.750000,-29.250000,-31.000000,-37.250000
50%,-13.50000,-17.000000,0.000000,-5.000000,-5.000000,-9.500000,-5.500000,-17.500000,-23.000000
75%,21.00000,10.250000,36.000000,19.500000,13.500000,16.250000,21.750000,12.500000,-1.000000
max,216.00000,161.000000,160.000000,157.000000,197.000000,439.000000,308.000000,202.000000,239.000000


In [306]:
# запишем полученный датафрейм в базу данных
migration.to_sql('migration', engine, if_exists='replace', index=False, chunksize=1000)

2020-09-01 22:28:32,407 INFO sqlalchemy.engine.base.Engine DESCRIBE `migration`
2020-09-01 22:28:32,409 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:32,412 INFO sqlalchemy.engine.base.Engine DESCRIBE `migration`
2020-09-01 22:28:32,412 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:32,416 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `healthcare_demographics`
2020-09-01 22:28:32,417 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:32,421 INFO sqlalchemy.engine.base.Engine SHOW CREATE TABLE `migration`
2020-09-01 22:28:32,422 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:32,427 INFO sqlalchemy.engine.base.Engine 
DROP TABLE migration
2020-09-01 22:28:32,427 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:32,441 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-01 22:28:32,456 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE migration (
	`Субъект` TEXT, 
	`2005` FLOAT(23), 
	`2010` FLOAT(23), 
	`2011` FLOAT(23), 
	`2012` FLOAT(23), 
	`20

<a id='section_id8'></a>

  ### <center><h2>[РАСХОДЫ КОНСОЛИДИРОВАННЫХ БЮДЖЕТОВ СУБЪЕКТОВ РОССИЙСКОЙ ФЕДЕРАЦИИ НА ЗДРАВООХРАНЕНИЕ)](#section_id2)   </h2> </center> 
                                                                                              
### <center><h4>(миллионов рублей)</h4></center>

In [307]:
budgeting = pd.read_excel("https://rosstat.gov.ru/bgd/regl/b19_34/IssWWW.exe/Stg/r_7.xls")
budgeting = budgeting.drop([0,1 , 2, 96])
budgeting.columns = ['Субъект','2013','2014','2015','2016','2017','2018']
budgeting.reset_index(inplace = True, drop = True)
budgeting

,Субъект,2013,2014,2015,2016,2017,2018
0,Центральный федеральный округ,NaN,NaN,NaN,NaN,NaN,NaN
1,Белгородская область,9979.057267,9718.915135,10552.197413,10154.478248,5931.359101,7824.680619
2,Брянская область,5718.159801,6627.482140,6825.480518,2299.673267,2850.557614,3959.374888
3,Владимирская область,8591.960727,8355.631947,8375.279328,8453.836968,5052.233141,6411.801821
4,Воронежская область,15818.075533,16192.858049,17124.904842,16460.749301,7698.299732,10249.083667
...,...,...,...,...,...,...,...
88,Амурская область,8098.232255,8454.750748,7292.900617,2292.428871,2949.882035,4457.616907
89,Магаданская область,5431.758454,5242.134132,5465.583009,5796.575863,3944.299949,5279.829166
90,Сахалинская область,12339.637255,15854.174164,15987.605950,18197.011661,12085.079900,14620.308127
91,Еврейская автономная область,2862.944789,1835.891068,2071.107986,1852.173194,580.727776,920.345344


#### Видим, что в федеральных округах значения отсутствуют, поэтому приведем таблицу общей форме.

In [308]:
def summ(a,b,b2,c,df):
    '''функция суммирует значения по федеральным округам и записывает их в таблицу.
       a - индекс в таблице значения федерального округа
       b - индекс региона от которого необходимо начать суммирование
       b2- индекс региона до которого необходимо просуммировать
       c - название федерального округа
       df- dataframe 
    '''
    df.iloc[a]  = df.iloc[b:b2].sum() 
    df.iloc[a,0] = c
    return df

In [309]:
summ(0,1,19,'Центральный федеральный округ', budgeting)
summ(19,20,31,'Северо-Западный федеральный округ', budgeting)
summ(31,32,40,'Южный федеральный округ', budgeting)
summ(40,41,48,'Северо-Кавказский федеральный округ', budgeting)
summ(48,49,63,'Приволжский федеральный округ', budgeting)
summ(63,64,70,'Уральский федеральный округ', budgeting)
summ(70,71,81,'Сибирский федеральный округ', budgeting)
summ(81,82,93,'Дальневосточный федеральный округ', budgeting)

,Субъект,2013,2014,2015,2016,2017,2018
0,Центральный федеральный округ,402245.495687,404904.594893,392331.412962,364948.909760,316524.565608,343671.695146
1,Белгородская область,9979.057267,9718.915135,10552.197413,10154.478248,5931.359101,7824.680619
2,Брянская область,5718.159801,6627.482140,6825.480518,2299.673267,2850.557614,3959.374888
3,Владимирская область,8591.960727,8355.631947,8375.279328,8453.836968,5052.233141,6411.801821
4,Воронежская область,15818.075533,16192.858049,17124.904842,16460.749301,7698.299732,10249.083667
...,...,...,...,...,...,...,...
88,Амурская область,8098.232255,8454.750748,7292.900617,2292.428871,2949.882035,4457.616907
89,Магаданская область,5431.758454,5242.134132,5465.583009,5796.575863,3944.299949,5279.829166
90,Сахалинская область,12339.637255,15854.174164,15987.605950,18197.011661,12085.079900,14620.308127
91,Еврейская автономная область,2862.944789,1835.891068,2071.107986,1852.173194,580.727776,920.345344


In [310]:
# проверим пропущенные значения
budgeting.isna().sum() 

Субъект    0
2013       2
2014       0
2015       0
2016       0
2017       0
2018       0
dtype: int64

In [311]:
budgeting[budgeting['2013'].isnull()]

,Субъект,2013,2014,2015,2016,2017,2018
34,Республика Крым,NaN,16588.241410,17458.8,17092.451577,13794.050926,13898.326246
39,г. Севастополь,NaN,2802.041503,3817.8,3978.907329,2682.477983,3751.507609


In [312]:
budgeting.fillna(0, inplace=True)

reduce_mem_usage(budgeting)

# посмотрим остались ли какие либо выпадающие значения
budgeting.describe()

Memory usage of dataframe is 0.01 MB
Memory usage after optimization is: 0.01 MB
Decreased by -7.9%


,2013,2014,2015,2016,2017,2018
count,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000
mean,26899.607422,28303.849609,29155.843750,27551.744141,18219.283203,20445.519531
std,55030.984375,56165.613281,55562.476562,53428.875000,43107.500000,45874.304688
min,0.000000,1801.167114,2071.107910,1641.283081,580.727783,636.048828
25%,5572.672363,6006.387207,6584.199707,5478.138672,2632.020020,3338.364502
50%,9432.289062,9751.922852,10552.197266,9656.916992,4656.175293,5964.617188
75%,19178.113281,19520.853516,20190.552734,21079.556641,10607.386719,13079.920898
max,402245.500000,404904.593750,392331.406250,364948.906250,316524.562500,343671.687500


In [313]:
# запишем полученный датафрейм в базу данных
budgeting.to_sql('budgeting', engine, if_exists='replace', index=False, chunksize=1000)

2020-09-01 22:28:35,613 INFO sqlalchemy.engine.base.Engine DESCRIBE `budgeting`
2020-09-01 22:28:35,615 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:35,619 INFO sqlalchemy.engine.base.Engine DESCRIBE `budgeting`
2020-09-01 22:28:35,620 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:35,623 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `healthcare_demographics`
2020-09-01 22:28:35,624 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:35,626 INFO sqlalchemy.engine.base.Engine SHOW CREATE TABLE `budgeting`
2020-09-01 22:28:35,627 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:35,631 INFO sqlalchemy.engine.base.Engine 
DROP TABLE budgeting
2020-09-01 22:28:35,633 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:35,645 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-01 22:28:35,648 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE budgeting (
	`Субъект` TEXT, 
	`2013` FLOAT(23), 
	`2014` FLOAT(23), 
	`2015` FLOAT(23), 
	`2016` FLOAT(23), 
	`20

<a id='section_id9'></a>

  ### <center><h2>[СРЕДНЕМЕСЯЧНАЯ НОМИНАЛЬНАЯ НАЧИСЛЕННАЯ ЗАРАБОТНАЯ ПЛАТА РАБОТНИКОВ ЗДРАВООХРАНЕНИЯ](#section_id2)</h2> </center> 
                                                                                              
### <center><h4>(рублей)</h4></center>

In [314]:
average_monthly_wage = pd.read_excel("https://rosstat.gov.ru/bgd/regl/b19_34/IssWWW.exe/Stg/r_4.xls",
                                     sheet_name='Т.4.2.')
average_monthly_wage = average_monthly_wage.drop([0,99, 100])
average_monthly_wage.columns = ['Субъект','2005','2010','2013','2014','2015','2016','2017','2018']
average_monthly_wage.reset_index(inplace = True, drop = True)
average_monthly_wage

,Субъект,2005,2010,2013,2014,2015,2016,2017,2018
0,Российская Федерация,5927,15794.0,24570.0,27213.0,28318.0,29882.0,31980.0,40027.0
1,Центральный федеральный округ,6673,20994.0,30749.0,33330.0,34730.0,36864.0,39650.0,50269.0
2,Белгородская область,4026,10198.0,17594.0,20177.0,22666.0,23988.0,25202.0,30150.0
3,Брянская область,3722,8705.0,15726.0,17854.0,18044.0,19343.0,20856.0,25499.0
4,Владимирская область,4969,12577.0,18768.0,21003.0,21697.0,22924.0,24921.0,30049.0
...,...,...,...,...,...,...,...,...,...
93,Амурская область,6646,15602.0,22406.0,27899.0,27936.0,27522.0,29532.0,40020.0
94,Магаданская область,10133,22811.0,43702.0,49713.0,52205.0,52646.0,57496.0,75977.0
95,Сахалинская область,9181,27400.0,40695.0,45993.0,48695.0,52779.0,57076.0,69272.0
96,Еврейская автономная область,6515,14290.0,23174.0,25736.0,27624.0,27706.0,29031.0,36191.0


In [315]:
average_monthly_wage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Субъект  98 non-null     object 
 1   2005     94 non-null     object 
 2   2010     94 non-null     float64
 3   2013     94 non-null     float64
 4   2014     94 non-null     float64
 5   2015     96 non-null     float64
 6   2016     96 non-null     float64
 7   2017     96 non-null     float64
 8   2018     96 non-null     float64
dtypes: float64(7), object(2)
memory usage: 7.0+ KB


In [316]:
# проверим пропущенные значения
average_monthly_wage.isna().sum() 

Субъект    0
2005       4
2010       4
2013       4
2014       4
2015       2
2016       2
2017       2
2018       2
dtype: int64

In [317]:
average_monthly_wage[average_monthly_wage['2010'].isnull()]

,Субъект,2005,2010,2013,2014,2015,2016,2017,2018
24,в том числе:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,Республика Крым,NaN,NaN,NaN,NaN,22263.0,23647.0,25167.0,29705.0
42,г. Севастополь,NaN,NaN,NaN,NaN,21008.0,23027.0,27129.0,33595.0
70,в том числе:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [318]:
# удалим пустые строки 
average_monthly_wage = average_monthly_wage.drop([24, 70])
average_monthly_wage.reset_index(inplace = True, drop = True)

In [319]:
average_monthly_wage.fillna(0, inplace=True)
average_monthly_wage.iloc[36:49]

,Субъект,2005,2010,2013,2014,2015,2016,2017,2018
36,Республика Крым,0,0.0,0.0,0.0,22263.0,23647.0,25167.0,29705.0
37,Краснодарский край,4878,12488.0,20114.0,22398.0,23382.0,24822.0,26471.0,32766.0
38,Астраханская область,3961,11160.0,19059.0,21121.0,21256.0,22029.0,23624.0,30745.0
39,Волгоградская область,4385,11442.0,17580.0,20138.0,21541.0,22861.0,24029.0,30049.0
40,Ростовская область,4118,11549.0,17517.0,20027.0,21140.0,22867.0,23869.0,30265.0
41,г. Севастополь,0,0.0,0.0,0.0,21008.0,23027.0,27129.0,33595.0
42,Северо-Кавказский федеральный округ,4055,10002.0,16468.0,18702.0,19734.0,20467.0,21761.0,26553.0
43,Республика Дагестан,3247,8683.0,14524.0,17414.0,18742.0,18852.0,20294.0,25220.0
44,Республика Ингушетия,4810,10453.0,16778.0,19323.0,19791.0,19658.0,20437.0,26500.0
45,Кабардино-Балкарская Республика,3796,8895.0,15407.0,18162.0,18279.0,19419.0,20596.0,24844.0


In [320]:
# заменим троеточие значением 0
average_monthly_wage = average_monthly_wage.replace(to_replace='…', value=0)

reduce_mem_usage(average_monthly_wage)

# посмотрим остались ли какие либо выпадающие значения
average_monthly_wage.describe()

Memory usage of dataframe is 0.01 MB
Memory usage after optimization is: 0.01 MB
Decreased by 8.6%


,2005,2010,2013,2014,2015,2016,2017,2018
count,96.00000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,5817.75000,14341.083008,22963.781250,25604.333984,27105.968750,28335.937500,30512.384766,37762.886719
std,2956.56591,6869.427734,10317.900391,11247.232422,10972.191406,11606.390625,12724.078125,15343.696289
min,0.00000,0.000000,0.000000,0.000000,16370.000000,16886.000000,19111.000000,24366.000000
25%,4047.75000,10440.000000,17301.750000,19185.250000,19954.000000,21088.250000,22731.000000,28430.500000
50%,4969.00000,12447.500000,19881.500000,22161.500000,23004.000000,23787.500000,25575.500000,32336.000000
75%,6525.25000,15607.750000,25373.250000,27646.250000,28737.750000,29417.000000,31887.750000,40083.250000
max,17573.00000,39960.000000,58521.000000,64545.000000,66821.000000,71017.000000,81746.000000,95663.000000


In [321]:
average_monthly_wage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   Субъект  96 non-null     category
 1   2005     96 non-null     int16   
 2   2010     96 non-null     float32 
 3   2013     96 non-null     float32 
 4   2014     96 non-null     float32 
 5   2015     96 non-null     float32 
 6   2016     96 non-null     float32 
 7   2017     96 non-null     float32 
 8   2018     96 non-null     float32 
dtypes: category(1), float32(7), int16(1)
memory usage: 6.3 KB


In [322]:
# запишем полученный датафрейм в базу данных
average_monthly_wage.to_sql('average_monthly_wage', engine, if_exists='replace', index=False, chunksize=1000)

2020-09-01 22:28:39,073 INFO sqlalchemy.engine.base.Engine DESCRIBE `average_monthly_wage`
2020-09-01 22:28:39,074 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:39,079 INFO sqlalchemy.engine.base.Engine DESCRIBE `average_monthly_wage`
2020-09-01 22:28:39,082 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:39,088 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `healthcare_demographics`
2020-09-01 22:28:39,089 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:39,092 INFO sqlalchemy.engine.base.Engine SHOW CREATE TABLE `average_monthly_wage`
2020-09-01 22:28:39,093 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:39,099 INFO sqlalchemy.engine.base.Engine 
DROP TABLE average_monthly_wage
2020-09-01 22:28:39,100 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:39,113 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-01 22:28:39,116 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE average_monthly_wage (
	`Субъект` TEXT, 
	`2005` BIGINT, 
	`2010` FLOA

<a id='section_id10'></a>

  ### <center><h2>[ЧИСЛО БОЛЬНИЧНЫХ КОЕК ПО СУБЪЕКТАМ РОССИЙСКОЙ ФЕДЕРАЦИИ](#section_id2)</h2> </center> 
                                                                                              
### <center><h4>(тысяч коек)</h4></center>

In [323]:
beds = pd.read_excel("https://rosstat.gov.ru/bgd/regl/b19_34/IssWWW.exe/Stg/r_3.xlsx",
                                     sheet_name='Т.3.2.')
beds = beds.drop([0,99])
beds.columns = ['Субъект','2005','2010','2011','2012','2013','2014','2015','2016','2017','2018']
beds.reset_index(inplace = True, drop = True)

In [324]:
beds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Субъект  98 non-null     object 
 1   2005     94 non-null     object 
 2   2010     94 non-null     object 
 3   2011     94 non-null     float64
 4   2012     94 non-null     float64
 5   2013     94 non-null     float64
 6   2014     96 non-null     float64
 7   2015     96 non-null     float64
 8   2016     96 non-null     float64
 9   2017     96 non-null     float64
 10  2018     96 non-null     float64
dtypes: float64(8), object(3)
memory usage: 8.5+ KB


In [325]:
# удалим пустые строки 
beds = beds.drop([24, 70])
beds.reset_index(inplace = True, drop = True)

In [326]:
beds.fillna(0, inplace=True)
beds.iloc[36:49]

,Субъект,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
36,Республика Крым,0,0,0.0,0.0,0.0,16.3,14.8,14.5,15.1,14.6
37,Краснодарский край,49.2,42.7,44.0,44.2,44.1,41.3,40.7,41.1,42.0,41.5
38,Астраханская область,11.7,10.3,10.8,10.4,10.0,9.8,9.6,9.4,9.0,9.0
39,Волгоградская область,31.5,28.4,27.7,26.9,24.7,22.6,22.8,22.2,22.0,22.0
40,Ростовская область,43.6,37.3,38.7,38.2,37.9,34.2,34.5,34.5,34.1,33.8
41,г. Севастополь,0,0,0.0,0.0,0.0,2.8,2.7,2.6,3.6,3.6
42,Северо-Кавказский федеральный округ,64.8,73.6,73.1,74.1,73.0,71.5,70.9,70.4,70.4,71.5
43,Республика Дагестан,18,19.7,20.3,20.7,20.9,20.7,20.5,20.4,21.2,21.5
44,Республика Ингушетия,1.9,2.1,2.1,2.1,2.1,2.3,2.3,2.3,2.2,2.2
45,Кабардино-Балкарская Республика,9,8.4,8.3,8.4,8.1,7.5,7.3,7.1,6.7,6.5


In [327]:
# заменим str 
beds = beds.replace(to_replace='…', value=0)
beds = beds.replace(to_replace='16, 5', value=16.5)
reduce_mem_usage(beds)

# посмотрим остались ли какие либо выпадающие значения
beds.describe()

Memory usage of dataframe is 0.01 MB
Memory usage after optimization is: 0.01 MB
Decreased by 13.8%


,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,49.734371,42.283325,42.515629,42.055202,41.103123,40.005215,38.593754,37.816662,37.354168,37.037502
std,169.114914,143.480148,144.291412,142.674866,139.367355,135.476089,130.511383,127.852119,126.238693,125.173271
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.400000,0.400000,0.400000
25%,9.775000,8.300000,8.225000,8.475000,8.175000,8.050000,7.750000,7.250000,7.175000,6.925000
50%,15.100000,12.000000,12.050000,12.100000,12.050000,11.800000,11.150000,10.850000,10.600000,10.500000
75%,31.125000,26.575000,27.125000,26.750000,25.900001,24.350000,24.525000,24.400000,23.800000,23.500000
max,1575.400024,1339.500000,1347.099976,1332.300049,1301.900024,1266.800049,1222.000000,1197.199951,1182.699951,1172.800049


In [328]:
beds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   Субъект  96 non-null     category
 1   2005     96 non-null     float32 
 2   2010     96 non-null     float32 
 3   2011     96 non-null     float32 
 4   2012     96 non-null     float32 
 5   2013     96 non-null     float32 
 6   2014     96 non-null     float32 
 7   2015     96 non-null     float32 
 8   2016     96 non-null     float32 
 9   2017     96 non-null     float32 
 10  2018     96 non-null     float32 
dtypes: category(1), float32(10)
memory usage: 7.2 KB


In [329]:
# запишем полученный датафрейм в базу данных
beds.to_sql('beds', engine, if_exists='replace', index=False, chunksize=1000)

2020-09-01 22:28:42,055 INFO sqlalchemy.engine.base.Engine DESCRIBE `beds`
2020-09-01 22:28:42,056 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:42,059 INFO sqlalchemy.engine.base.Engine DESCRIBE `beds`
2020-09-01 22:28:42,060 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:42,063 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `healthcare_demographics`
2020-09-01 22:28:42,064 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:42,069 INFO sqlalchemy.engine.base.Engine SHOW CREATE TABLE `beds`
2020-09-01 22:28:42,070 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:42,075 INFO sqlalchemy.engine.base.Engine 
DROP TABLE beds
2020-09-01 22:28:42,075 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:42,090 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-01 22:28:42,096 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE beds (
	`Субъект` TEXT, 
	`2005` FLOAT(23), 
	`2010` FLOAT(23), 
	`2011` FLOAT(23), 
	`2012` FLOAT(23), 
	`2013` FLOAT(23), 
	`2014` F

<a id='section_id11'></a>

  ### <center><h2>[КОНТИНГЕНТЫ ПАЦИЕНТОВ С АЛКОГОЛИЗМОМ И АЛКОГОЛЬНЫМИ ПСИХОЗАМИ ПО СУБЪКТАМ РОССИЙСКОЙ ФЕДЕРАЦИИ](#section_id2)</h2> </center> 
                                                                                              
### <center><h4>(Всего, человек)</h4></center>

In [330]:
alcoholism = pd.read_excel("https://rosstat.gov.ru/bgd/regl/b19_34/IssWWW.exe/Stg/r_2.xls",
                                     sheet_name='Т.2.8.')
alcoholism.drop(['Unnamed: 9','Unnamed: 10','Unnamed: 11','Unnamed: 12','Unnamed: 13',
                 'Unnamed: 14','Unnamed: 15','Unnamed: 16'], axis=1, inplace=True)
alcoholism = alcoholism.drop([0,1])
alcoholism.columns = ['Субъект','2005','2010','2013','2014','2015','2016','2017','2018']
alcoholism.reset_index(inplace = True, drop = True)
alcoholism

,Субъект,2005,2010,2013,2014,2015,2016,2017,2018
0,Российская Федерация,2190674,1953129.0,1746514.0,1689975.0,1577046.0,1444520.0,1304613.0,1208582.0
1,Центральный федеральный округ,614230,555531.0,516389.0,501599.0,470280.0,421168.0,376976.0,358364.0
2,Белгородская область,19686,16252.0,14567.0,13939.0,12787.0,12127.0,11602.0,10770.0
3,Брянская область,31897,29291.0,28032.0,27111.0,26098.0,24254.0,22666.0,21383.0
4,Владимирская область,29586,26879.0,25101.0,24340.0,23323.0,22573.0,22151.0,21196.0
...,...,...,...,...,...,...,...,...,...
93,Амурская область,14180,15025.0,11715.0,10948.0,10473.0,9848.0,8611.0,8306.0
94,Магаданская область,8810,6279.0,4398.0,3900.0,3779.0,3589.0,3341.0,2826.0
95,Сахалинская область,20345,13350.0,11573.0,11291.0,10941.0,10146.0,8889.0,8357.0
96,Еврейская автономная область,1938,2665.0,2637.0,2653.0,2602.0,2788.0,2744.0,2417.0


In [331]:
# проверим пропущенные значения
alcoholism.isna().sum() 

Субъект    0
2005       4
2010       4
2013       4
2014       2
2015       2
2016       2
2017       2
2018       2
dtype: int64

In [332]:
# удалим пустые строки 
alcoholism = alcoholism.drop([24, 70])
alcoholism.reset_index(inplace = True, drop = True)

In [333]:
alcoholism.fillna(0, inplace=True)
alcoholism.iloc[36:49]

,Субъект,2005,2010,2013,2014,2015,2016,2017,2018
36,Республика Крым,0,0.0,0.0,22127.0,22118.0,21852.0,21425.0,20372.0
37,Краснодарский край,81175,68101.0,47929.0,43483.0,38223.0,31706.0,26585.0,20673.0
38,Астраханская область,16358,15024.0,12304.0,11044.0,10161.0,9235.0,7548.0,6551.0
39,Волгоградская область,41269,30544.0,27874.0,26924.0,21069.0,16603.0,15259.0,14623.0
40,Ростовская область,60488,57049.0,51307.0,49019.0,46741.0,44462.0,42187.0,40362.0
41,г. Севастополь,0,0.0,0.0,3466.0,3300.0,2982.0,2686.0,2451.0
42,Северо-Кавказский федеральный округ,56920,58392.0,53597.0,52533.0,51645.0,49507.0,47191.0,46367.0
43,Республика Дагестан,9526,10800.0,9515.0,8559.0,8063.0,7498.0,7409.0,7346.0
44,Республика Ингушетия,71,42.0,43.0,43.0,41.0,43.0,41.0,40.0
45,Кабардино-Балкарская Республика,7251,6997.0,6153.0,6244.0,6253.0,6208.0,6033.0,5950.0


In [334]:
# заменим троеточие значением 0
alcoholism = alcoholism.replace(to_replace='…', value=0)

reduce_mem_usage(alcoholism)

# посмотрим остались ли какие либо выпадающие значения
alcoholism.describe()

Memory usage of dataframe is 0.01 MB
Memory usage after optimization is: 0.01 MB
Decreased by 5.9%


,2005,2010,2013,2014,2015,2016,2017,2018
count,9.600000e+01,9.600000e+01,9.600000e+01,9.600000e+01,9.600000e+01,9.600000e+01,9.600000e+01,9.600000e+01
mean,6.923909e+04,6.169467e+04,5.514752e+04,5.335843e+04,4.981868e+04,4.560931e+04,4.119406e+04,3.814600e+04
std,2.358371e+05,2.105920e+05,1.890078e+05,1.828060e+05,1.705171e+05,1.561608e+05,1.409622e+05,1.306316e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,4.300000e+01,4.100000e+01,4.300000e+01,4.100000e+01,4.000000e+01
25%,1.341850e+04,1.230200e+04,1.117925e+04,1.086300e+04,1.039500e+04,9.356500e+03,8.345250e+03,8.066000e+03
50%,2.203050e+04,2.018750e+04,1.790500e+04,1.768000e+04,1.682550e+04,1.596000e+04,1.306150e+04,1.221600e+04
75%,3.987700e+04,3.501650e+04,3.332100e+04,3.181725e+04,2.946025e+04,2.523000e+04,2.296325e+04,2.124275e+04
max,2.190674e+06,1.953129e+06,1.746514e+06,1.689975e+06,1.577046e+06,1.444520e+06,1.304613e+06,1.208582e+06


In [335]:
alcoholism.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   Субъект  96 non-null     category
 1   2005     96 non-null     int32   
 2   2010     96 non-null     float32 
 3   2013     96 non-null     float32 
 4   2014     96 non-null     float32 
 5   2015     96 non-null     float32 
 6   2016     96 non-null     float32 
 7   2017     96 non-null     float32 
 8   2018     96 non-null     float32 
dtypes: category(1), float32(7), int32(1)
memory usage: 6.5 KB


In [336]:
# запишем полученный датафрейм в базу данных
alcoholism.to_sql('alcoholism', engine, if_exists='replace', index=False, chunksize=1000)

2020-09-01 22:28:46,494 INFO sqlalchemy.engine.base.Engine DESCRIBE `alcoholism`
2020-09-01 22:28:46,495 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:46,498 INFO sqlalchemy.engine.base.Engine DESCRIBE `alcoholism`
2020-09-01 22:28:46,498 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:46,501 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `healthcare_demographics`
2020-09-01 22:28:46,502 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:46,505 INFO sqlalchemy.engine.base.Engine SHOW CREATE TABLE `alcoholism`
2020-09-01 22:28:46,507 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:46,511 INFO sqlalchemy.engine.base.Engine 
DROP TABLE alcoholism
2020-09-01 22:28:46,512 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:46,524 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-01 22:28:46,528 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE alcoholism (
	`Субъект` TEXT, 
	`2005` INTEGER, 
	`2010` FLOAT(23), 
	`2013` FLOAT(23), 
	`2014` FLOAT(23), 
	

<a id='section_id12'></a>

  ### <center><h2>[ПРЕРЫВАНИЕ БЕРЕМЕННОСТИ (АБОРТЫ)](#section_id2)</h2> </center> 
                                                                                              
### <center><h4>(На 1000 женщин 15 – 49 лет)</h4></center>

In [337]:
# скачаем файл и передадим его в функцию которая его обработает

abortion_url = 'https://rosstat.gov.ru/bgd/regl/b19_14p/IssWWW.exe/Stg/d01/06-07.docx'
abortion = read_docx(abortion_url)

# из полученных файлов сделаем датафреймы и и объединим их
df_0 = pd.DataFrame(abortion[0])
df_1 = pd.DataFrame(abortion[1])
abortion = df_0.append(df_1).reset_index(drop=True)

#обозначим названия столбцов и удалим ненужные строки
abortion.columns = ['Субъект','2005','2010','2014', '2015','2016','2017','2018','1','2','3', '4','5','6','7']
abortion.drop(['1','2','3', '4','5','6','7'], axis=1, inplace=True)
abortion.drop([0,1,26, 53,54,74], inplace = True)

abortion.reset_index(inplace = True, drop = True)
abortion

,Субъект,2005,2010,2014,2015,2016,2017,2018
0,Российская Федерация,43,32,26.0,24.0,24.0,22.0,19.0
1,Центральный федеральный округ,34,24,19.0,18.0,18.0,17.0,14.0
2,Белгородская область,31,19,14.0,14.0,14.0,13.0,11.0
3,Брянская область,47,37,31.0,32.0,29.0,26.0,24.0
4,Владимирская область,46,34,29.0,27.0,26.0,24.0,21.0
...,...,...,...,...,...,...,...,...
91,Амурская область,88,55,45.0,37.0,35.0,31.0,27.0
92,Магаданская область,61,55,50.0,46.0,45.0,39.0,32.0
93,Сахалинская область,67,54,47.0,41.0,41.0,38.0,33.0
94,Еврейская автономная область,66,53,45.0,47.0,44.0,43.0,38.0


In [338]:
# проверим пропущенные значения
abortion.isna().sum() 

Субъект    0
2005       2
2010       2
2014       0
2015       0
2016       0
2017       0
2018       0
dtype: int64

In [339]:
abortion.fillna(0, inplace=True)
abortion = abortion.replace(to_replace='…', value=0)
abortion.iloc[36:49]

,Субъект,2005,2010,2014,2015,2016,2017,2018
36,Республика Крым,0,0,20.0,21.0,21.0,18.0,16.0
37,Краснодарский край,40,24,20.0,19.0,19.0,15.0,13.0
38,Астраханская область,42,26,20.0,18.0,19.0,20.0,17.0
39,Волгоградская область,49,38,31.0,30.0,27.0,19.0,16.0
40,Ростовская область,31,24,16.0,14.0,15.0,14.0,12.0
41,г. Севастополь,0,0,24.0,23.0,21.0,20.0,16.0
42,Северо-Кавказскийфедеральный округ,25,17,14.0,13.0,13.0,12.0,11.0
43,Республика Дагестан,16,12,8.0,8.0,9.0,7.0,7.0
44,Республика Ингушетия,8,13,9.0,8.0,8.0,7.0,6.0
45,Кабардино-Балкарская Республика,26,14,12.0,11.0,11.0,10.0,8.0


In [340]:
# некоторые переменные являются категориальными приведем их в числовой вид
abortion['2005'] = pd.to_numeric(abortion['2005'])
abortion['2010'] = pd.to_numeric(abortion['2010'])

In [341]:
reduce_mem_usage(abortion)

# посмотрим остались ли какие либо выпадающие значения
abortion.describe()

Memory usage of dataframe is 0.01 MB
Memory usage after optimization is: 0.01 MB
Decreased by 9.7%


,2005,2010,2014,2015,2016,2017,2018
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,45.395833,34.947917,29.385416,27.031250,26.864584,25.187500,21.479166
std,15.214591,12.307781,10.067530,9.127694,9.173900,8.925791,7.368466
min,0.000000,0.000000,8.000000,8.000000,8.000000,7.000000,6.000000
25%,39.000000,26.750000,22.000000,20.000000,20.000000,19.000000,16.000000
50%,46.000000,35.000000,29.000000,27.000000,25.500000,24.000000,20.500000
75%,55.000000,44.250000,37.000000,34.250000,34.250000,33.000000,27.250000
max,88.000000,58.000000,53.000000,47.000000,46.000000,48.000000,38.000000


In [342]:
# запишем полученный датафрейм в базу данных
abortion.to_sql('abortion', engine, if_exists='replace', index=False, chunksize=1000)

2020-09-01 22:28:49,175 INFO sqlalchemy.engine.base.Engine DESCRIBE `abortion`
2020-09-01 22:28:49,177 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:49,184 INFO sqlalchemy.engine.base.Engine DESCRIBE `abortion`
2020-09-01 22:28:49,184 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:49,191 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `healthcare_demographics`
2020-09-01 22:28:49,192 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:49,196 INFO sqlalchemy.engine.base.Engine SHOW CREATE TABLE `abortion`
2020-09-01 22:28:49,197 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:49,203 INFO sqlalchemy.engine.base.Engine 
DROP TABLE abortion
2020-09-01 22:28:49,205 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:49,218 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-01 22:28:49,224 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE abortion (
	`Субъект` TEXT, 
	`2005` BIGINT, 
	`2010` BIGINT, 
	`2014` FLOAT(23), 
	`2015` FLOAT(23), 
	`2016` FLOAT(2

#### Сохраним все датасеты в формат excel :

In [343]:
population.to_excel('Население субъектов Российской Федерациина 1 января (тысяч человек).xls')
mortality.to_excel('ОБЩИЕ КОЭФФИЦИЕНТЫ СМЕРТНОСТИ (число умерших на 1000 человек населения).xls')
birthrate.to_excel('ОБЩИЕ КОЭФФИЦИЕНТЫ РОЖДАЕМОСТИ (число родившихся на 1000 человек населения).xls')
marriage_rate.to_excel('ОБЩИЕ КОЭФФИЦИЕНТЫ БРАЧНОСТИ (на 1000 человек населения).xls')
divorce_rate.to_excel('ОБЩИЕ КОЭФФИЦИЕНТЫ РАЗВОДИМОСТИ (на 1000 человек населения).xls')
migration.to_excel('КОЭФФИЦИЕНТЫ МИГРАЦИОННОГО ПРИРОСТА) (на 10 000 человек населения).xls')
budgeting.to_excel('РАСХОДЫ КОНСОЛИДИРОВАННЫХ БЮДЖЕТОВ СУБЪЕКТОВ РОССИЙСКОЙ ФЕДЕРАЦИИ НА ЗДРАВООХРАНЕНИЕ)(миллионов рублей).xls')
average_monthly_wage.to_excel('СРЕДНЕМЕСЯЧНАЯ НОМИНАЛЬНАЯ НАЧИСЛЕННАЯ ЗАРАБОТНАЯ ПЛАТА РАБОТНИКОВ ЗДРАВООХРАНЕНИЯ(рублей).xls')
beds.to_excel('ЧИСЛО БОЛЬНИЧНЫХ КОЕК ПО СУБЪЕКТАМ РОССИЙСКОЙ ФЕДЕРАЦИИ(тысяч коек).xls')
alcoholism.to_excel('КОНТИНГЕНТЫ ПАЦИЕНТОВ С АЛКОГОЛИЗМОМ И АЛКОГОЛЬНЫМИ ПСИХОЗАМИ ПО СУБЪКТАМ РОССИЙСКОЙ ФЕДЕРАЦИИ(Всего, человек).xls')
abortion.to_excel('ПРЕРЫВАНИЕ БЕРЕМЕННОСТИ (АБОРТЫ).xls')

In [344]:
# запрос к созданной базе данных
sql_df = pd.read_sql(
    "SELECT * FROM population",
    con=engine
)
sql_df

2020-09-01 22:28:49,496 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM population`
2020-09-01 22:28:49,500 INFO sqlalchemy.engine.base.Engine {}
2020-09-01 22:28:49,520 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-09-01 22:28:49,523 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM population
2020-09-01 22:28:49,524 INFO sqlalchemy.engine.base.OptionEngine {}


,Субъект,2000,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Российская Федерация,146890.0,143801.0,142833.0,142865.0,143056.0,143347.0,143667.0,146267.0,146545.0,146804.0,146880.0,146781.0
1,Центральный федеральный округ,38228.0,38044.0,38335.0,38445.0,38538.0,38679.0,38820.0,38951.0,39104.0,39209.0,39311.0,39378.0
2,Белгородская область,1502.0,1512.0,1532.0,1532.0,1536.0,1541.0,1544.0,1548.0,1550.0,1553.0,1550.0,1548.0
3,Брянская область,1423.0,1344.0,1287.0,1275.0,1264.0,1254.0,1242.0,1233.0,1226.0,1221.0,1211.0,1200.0
4,Владимирская область,1576.0,1498.0,1450.0,1441.0,1432.0,1422.0,1413.0,1406.0,1397.0,1390.0,1378.0,1366.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Амурская область,936.0,874.0,835.0,829.0,821.0,817.0,811.0,810.0,806.0,802.0,798.0,794.0
92,Магаданская область,202.0,174.0,159.0,156.0,155.0,152.0,150.0,148.0,147.0,146.0,144.0,141.0
93,Сахалинская область,569.0,530.0,501.0,497.0,495.0,494.0,491.0,488.0,487.0,487.0,490.0,490.0
94,Еврейская автономная область,195.0,186.0,178.0,176.0,175.0,173.0,171.0,169.0,166.0,164.0,162.0,160.0
